In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

cust_df = pd.read_csv("./kaggle_data/santander-customer-satisfaction/train.csv", encoding='latin-1')
print('dataset shape:', cust_df.shape)
cust_df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


산탄데르 은행 고객 만족도조사 결과
피처 - 익명화, 컬럼명 익명화, 수치, 0인 데이터가 대다수 - 희소성
타겟 - TARGET

# 데이터 탐색

In [2]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


100명중 4명이 불만족하는 데이터

In [2]:
cust_df['TARGET'].value_counts() #0 만족, 1 불만족
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()

print(f'불만족 비율은 : {unsatisfied_cnt/total_cnt:.2f}')

불만족 비율은 : 0.04


최소값, Q1, Q2, Q3 가 0인 피처가 많다.
최소값 -999999.000000 인 피처 var3 > Nan > 중위값/최빈값 2로 대체

In [5]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [6]:
cust_df.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
           ...  
 231           1
 188           1
 168           1
 135           1
 87            1
Name: var3, Length: 208, dtype: int64

## 피처 선택 - id 삭제, var3 은 최빈값 2로 대체

In [3]:
cust_df.drop('ID', axis=1, inplace=True)
cust_df['var3'].replace(-999999, 2, inplace=True)

cust_df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


## X, y 데이터 분할 


In [4]:
X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]
X_features.shape, y_labels.shape

((76020, 369), (76020,))

## 훈련/테스트 분할

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
#X_train.count()
print(f'학습 데이터 shape : {X_train.shape}, 테스트 데이터 shape : {X_test.shape}')

train_cnt = y_train.count()
test_cnt = y_test.count()
print(f'학습 데이터 값의 비율 : {y_train.value_counts()/train_cnt}')
print(f'테스트 데이터  값의 비율 : {y_test.value_counts()/test_cnt}')

학습 데이터 shape : (60816, 369), 테스트 데이터 shape : (15204, 369)
학습 데이터 값의 비율 : 0    0.960964
1    0.039036
Name: TARGET, dtype: float64
테스트 데이터  값의 비율 : 0    0.9583
1    0.0417
Name: TARGET, dtype: float64


# DT 예측기

# RF 예측기

# XGBoost 예측기 p271

## 훈련 데이터 > 훈련:검증 분할

In [7]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

## XGBoost model

In [8]:
%%time

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156)
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

[14:49:09] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.82179	validation_1-auc:0.80068
[1]	validation_0-auc:0.83092	validation_1-auc:0.80941
[2]	validation_0-auc:0.83207	validation_1-auc:0.80903
[3]	validation_0-auc:0.83288	validation_1-auc:0.80889
[4]	validation_0-auc:0.83414	validation_1-auc:0.80924
[5]	validation_0-auc:0.83524	validation_1-auc:0.80907
[6]	validation_0-auc:0.83568	validation_1-auc:0.81005
[7]	validation_0-auc:0.83741	validation_1-auc:0.81088
[8]	validation_0-auc:0.83896	validation_1-auc:0.81305
[9]	validation_0-auc:0.83949	validation_1-auc:0.81363
[10]	validation_0-auc:0.83908	validation_1-auc:0.81277
[11]	v

0.8397812474965844

In [9]:
### XGBoost 하이퍼 파리미터 최적화
from hyperopt import hp
from hyperopt import fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll.base import scope

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np

In [10]:
# 검색 공간
search_space = {
    'n_estimator' : scope.int(hp.quniform('n_estimator', 50, 300, 10))
    , 'max_depth' : scope.int(hp.quniform('max_depth', 5, 20, 1))
    , 'learning_rate' : hp.uniform('learning_rate', 0.01, 0.3) # 지수로 뽑음
    , 'subsample' : hp.uniform('subsample', 0.5, 1.0)
    , 'colsample_bytree' :hp.uniform('colsample_bytree', 0.5, 1.0)
}

In [11]:
def objective_func_xgb(params):
    model = XGBClassifier(
        n_estimator = 100 #params['n_estimator']
        , max_depth = params['max_depth']
        , learning_rate = params['learning_rate']
        #, subsample = params['subsample']
        , colsample_bytree = params['colsample_bytree']
        , random_state = 42
        , eval_metric = 'logloss'
    )

    ### k-Fold 방식 추가해보기

    score_mean = cross_val_score(model
                                , X_train
                                , y_train
                                , cv = 5
                                , scoring = 'accuracy'
                                ).mean()
    
    return {'loss' : -1 * score_mean, 'status' : STATUS_OK}
                   


In [13]:
trial_val = Trials()

best_01 = fmin(fn=objective_func_xgb
               , space = search_space
               , algo = tpe.suggest
               , max_evals = 50
               , trials = trial_val
               , rstate = np.random.default_rng(seed = 0)
               )

[14:49:27] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:49:36] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:49:45] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by lan

In [14]:
best_01

{'colsample_bytree': 0.6406824979336891,
 'learning_rate': 0.12212214378271394,
 'max_depth': 5.0,
 'n_estimator': 230.0,
 'subsample': 0.6796628299126117}

In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [55]:
best_model = XGBClassifier(
    n_estimators = int(best_01['n_estimator'])
    , max_depth = int(best_01['max_depth'])
    , learning_rate = best_01['learning_rate']
    , subsample = best_01['subsample']
    , closample_bytree = best_01['colsample_bytree']
    , random_state = 42
    , eval_metric = 'logloss'                       
)

eval_list = [(X_tr, y_tr), (X_val, y_val)]
best_model.fit(X_tr
               , y_tr
               , early_stopping_rounds = 100
               , eval_set = eval_list
               , eval_metric = 'auc'
               )

wr_pred = best_model.predict(X_test)
wr_pred_proba = best_model.predict_proba(X_test)[:, 1]

[16:21:57] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "closample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.81594	validation_1-auc:0.79980
[1]	validation_0-auc:0.82239	validation_1-auc:0.80414
[2]	validation_0-auc:0.82849	validation_1-auc:0.80826
[3]	validation_0-auc:0.83320	validation_1-auc:0.80867
[4]	validation_0-auc:0.83580	validation_1-auc:0.81107
[5]	validation_0-auc:0.83559	validation_1-auc:0.81076
[6]	validation_0-auc:0.83783	validation_1-auc:0.81346
[7]	validation_0-auc:0.83971	validation_1-auc:0.81422
[8]	validation_0-auc:0.84113	validation_1-auc:0.81373
[9]	validation_0-auc:0.84467	validation_1-auc:0.81816
[10]	validation_0-auc:0.84570	validation_1-auc:0.81923
[

In [56]:
print(f'test1 : {roc_auc_score(y_test, wr_pred)}')
print(f'test2 : {roc_auc_score(y_test, wr_pred_proba)}')

test1 : 0.5030859399526705
test2 : 0.8409906272124779


In [21]:
best_01

{'colsample_bytree': 0.6406824979336891,
 'learning_rate': 0.12212214378271394,
 'max_depth': 5.0,
 'n_estimator': 230.0,
 'subsample': 0.6796628299126117}

In [24]:
def get_clf_eval(y_test, pred=None, pred_probs=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)

    print(confusion)
    print('*'*20)
    print(f'''
          accuracy : {accuracy}
          precision : {precision}
          recall : {recall}
          f1 : {f1}
          roc_auc : {roc_auc}
          ''')
    

In [27]:
### LGBM 추가

lgbm_wrapper = LGBMClassifier(n_estimators= 400
                            , learning_rate = 0.05)

evals = [(X_tr, y_tr), (X_val, y_val)]
lgbm_wrapper.fit(X_tr, y_tr
                 , early_stopping_rounds=50
                 , eval_metric = 'logloss'
                 , eval_set = evals
                 , verbose = True
                 )

preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]


get_clf_eval(y_test, preds, pred_proba)

[1]	training's binary_logloss: 0.159747	valid_1's binary_logloss: 0.161487
[2]	training's binary_logloss: 0.155995	valid_1's binary_logloss: 0.158489
[3]	training's binary_logloss: 0.152875	valid_1's binary_logloss: 0.155925
[4]	training's binary_logloss: 0.150255	valid_1's binary_logloss: 0.153849
[5]	training's binary_logloss: 0.148052	valid_1's binary_logloss: 0.152054
[6]	training's binary_logloss: 0.146074	valid_1's binary_logloss: 0.1505
[7]	training's binary_logloss: 0.144325	valid_1's binary_logloss: 0.149162
[8]	training's binary_logloss: 0.142724	valid_1's binary_logloss: 0.14795
[9]	training's binary_logloss: 0.141297	valid_1's binary_logloss: 0.146862
[10]	training's binary_logloss: 0.139983	valid_1's binary_logloss: 0.145886
[11]	training's binary_logloss: 0.138763	valid_1's binary_logloss: 0.144993
[12]	training's binary_logloss: 0.137652	valid_1's binary_logloss: 0.14414
[13]	training's binary_logloss: 0.136616	valid_1's binary_logloss: 0.1434
[14]	training's binary_logl

In [37]:
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, pred_proba)


0.8403631765717119

In [44]:
### LGBM 추가

lgbm_wrapper = LGBMClassifier(n_estimators= 400
                            , learning_rate = 0.01)

evals = [(X_tr, y_tr), (X_val, y_val)]
lgbm_wrapper.fit(X_tr, y_tr
                 , early_stopping_rounds=50
                 , eval_metric = 'logloss'
                 , eval_set = evals
                 , verbose = True
                 )
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, pred_proba)


[1]	training's binary_logloss: 0.163593	valid_1's binary_logloss: 0.164704
[2]	training's binary_logloss: 0.162662	valid_1's binary_logloss: 0.163935
[3]	training's binary_logloss: 0.16176	valid_1's binary_logloss: 0.163194
[4]	training's binary_logloss: 0.160905	valid_1's binary_logloss: 0.162479
[5]	training's binary_logloss: 0.160099	valid_1's binary_logloss: 0.161808
[6]	training's binary_logloss: 0.159329	valid_1's binary_logloss: 0.161157
[7]	training's binary_logloss: 0.158585	valid_1's binary_logloss: 0.160552
[8]	training's binary_logloss: 0.157878	valid_1's binary_logloss: 0.159974
[9]	training's binary_logloss: 0.157174	valid_1's binary_logloss: 0.159406
[10]	training's binary_logloss: 0.156491	valid_1's binary_logloss: 0.158849
[11]	training's binary_logloss: 0.15584	valid_1's binary_logloss: 0.158318
[12]	training's binary_logloss: 0.155221	valid_1's binary_logloss: 0.157809
[13]	training's binary_logloss: 0.154618	valid_1's binary_logloss: 0.157322
[14]	training's binary_

0.8401193845007999

In [45]:
### LGBM 추가

lgbm_wrapper = LGBMClassifier(n_estimators= 500
                            , learning_rate = 0.01)

evals = [(X_tr, y_tr), (X_val, y_val)]
lgbm_wrapper.fit(X_tr, y_tr
                 , early_stopping_rounds=50
                 , eval_metric = 'logloss'
                 , eval_set = evals
                 , verbose = True
                 )
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, pred_proba)


[1]	training's binary_logloss: 0.163593	valid_1's binary_logloss: 0.164704
[2]	training's binary_logloss: 0.162662	valid_1's binary_logloss: 0.163935
[3]	training's binary_logloss: 0.16176	valid_1's binary_logloss: 0.163194
[4]	training's binary_logloss: 0.160905	valid_1's binary_logloss: 0.162479
[5]	training's binary_logloss: 0.160099	valid_1's binary_logloss: 0.161808
[6]	training's binary_logloss: 0.159329	valid_1's binary_logloss: 0.161157
[7]	training's binary_logloss: 0.158585	valid_1's binary_logloss: 0.160552
[8]	training's binary_logloss: 0.157878	valid_1's binary_logloss: 0.159974
[9]	training's binary_logloss: 0.157174	valid_1's binary_logloss: 0.159406
[10]	training's binary_logloss: 0.156491	valid_1's binary_logloss: 0.158849
[11]	training's binary_logloss: 0.15584	valid_1's binary_logloss: 0.158318
[12]	training's binary_logloss: 0.155221	valid_1's binary_logloss: 0.157809
[13]	training's binary_logloss: 0.154618	valid_1's binary_logloss: 0.157322
[14]	training's binary_

0.8403883460461733

In [75]:
### LGBM 추가
from hyperopt import hp

search_space = {
    'n_estimators' : scope.int(hp.quniform('n_estimators', 50, 300, 10))
    , 'max_depth' : scope.int(hp.quniform('max_depth', 5, 20, 1))
    , 'learning_rate' : hp.uniform('learning_rate', 0.01, 0.3)
    , 'subsample' : hp.uniform('subsample', 0.5, 1.0)
    , 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1.0)

}

def objective_func_lgbm(params):
    model = LGBMClassifier(
        n_estimators = params['n_estimators']
        , max_depth = params['max_depth']
        , learning_rate = params['learning_rate']
        , subsample = params['subsample']
        , colsample_bytree = params['colsample_bytree']
        , random_state = 42
        , eval_metric = 'logloss'
    )

    score_mean = cross_val_score(model
                                , X_train
                                , y_train
                                , cv = 5
                                , scoring = 'accuracy'
                                ).mean()
    
    return {'loss' : -1 * score_mean, 'status' : STATUS_OK}

trials = Trials()

best_params_lgbm = fmin(
    fn = objective_func_lgbm
    , space = search_space
    , algo = tpe.suggest
    , max_evals = 50
    , trials = trials
)

best_params_lgbm


[LightGBM] [Warning] Unknown parameter: eval_metric   
[LightGBM] [Warning] Unknown parameter: eval_metric   
[LightGBM] [Warning] Unknown parameter: eval_metric   
[LightGBM] [Warning] Unknown parameter: eval_metric   
[LightGBM] [Warning] Unknown parameter: eval_metric   
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                              
[LightGBM] [Warning] Unknown parameter: eval_metric                  

{'colsample_bytree': 0.6927676666640397,
 'learning_rate': 0.019374568079732202,
 'max_depth': 19.0,
 'n_estimators': 110.0,
 'subsample': 0.605538798755866}

In [81]:
best_params_lgbm['n_estimators']

110.0

In [86]:
lgbm_wrapper = LGBMClassifier(n_estimators= int(best_params_lgbm['n_estimators'])
                            , learning_rate = best_params_lgbm['learning_rate'])

evals = [(X_tr, y_tr), (X_val, y_val)]
lgbm_wrapper.fit(X_tr, y_tr
                 , early_stopping_rounds=80
                 , eval_metric = 'logloss'
                 , eval_set = evals
                 , verbose = True
                 )
pred_proba_lgbm = lgbm_wrapper.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, pred_proba_lgbm)

[1]	training's binary_logloss: 0.162676	valid_1's binary_logloss: 0.163935
[2]	training's binary_logloss: 0.160927	valid_1's binary_logloss: 0.162528
[3]	training's binary_logloss: 0.159364	valid_1's binary_logloss: 0.161205
[4]	training's binary_logloss: 0.157936	valid_1's binary_logloss: 0.160011
[5]	training's binary_logloss: 0.156594	valid_1's binary_logloss: 0.158921
[6]	training's binary_logloss: 0.155326	valid_1's binary_logloss: 0.157887
[7]	training's binary_logloss: 0.154159	valid_1's binary_logloss: 0.15695
[8]	training's binary_logloss: 0.153066	valid_1's binary_logloss: 0.156044
[9]	training's binary_logloss: 0.152032	valid_1's binary_logloss: 0.155224
[10]	training's binary_logloss: 0.151064	valid_1's binary_logloss: 0.154467
[11]	training's binary_logloss: 0.150139	valid_1's binary_logloss: 0.153728
[12]	training's binary_logloss: 0.149252	valid_1's binary_logloss: 0.153007
[13]	training's binary_logloss: 0.148426	valid_1's binary_logloss: 0.152346
[14]	training's binary

0.8402565446046388

### XGBM vs LGBM 평가
1. AUC 기준 평가 결과
2. 각각의 파라미터값

In [69]:
#XGBM
print(f'''
auc : {roc_auc_score(y_test, pred_proba)}
n_estimators : {best_model.get_params()['n_estimators']}
learning_rate : {best_model.get_params()['learning_rate']}
      ''')



auc : 0.840595060504169
n_estimators : 230
learning_rate : 0.12212214378271394
      


In [91]:
#LGBM
print(f'''
auc : {roc_auc_score(y_test, pred_proba_lgbm)}
n_estimators : {best_params_lgbm['n_estimators']}
learning_rate : {best_params_lgbm['learning_rate']}

xgbm이 더큰가? {roc_auc_score(y_test, pred_proba) > roc_auc_score(y_test, pred_proba_lgbm)}
      ''')



auc : 0.8402565446046388
n_estimators : 110.0
learning_rate : 0.019374568079732202

xgbm이 더큰가? True
      


- LGBM  
auc : 0.840595060504169  
n_estimators : 500  
learning_rate : 0.01  
early_stopping_rounds : 80  

# 하이퍼오피티, LGBM 추가 예정

# 최적화 파라미터 기준 모델 평가

n_estimator=500
learning_rate = 0.15
max_depth = 5
